In [24]:
import sys
print("python",sys.version)
import torch
print("torch",torch.__version__)
print("cuda",torch.version.cuda)
print(torch.cuda.is_available())

python 3.9.12 (tags/v3.9.12:b28265d, Mar 23 2022, 23:52:46) [MSC v.1929 64 bit (AMD64)]
torch 2.1.2+cu121
cuda 12.1
True


In [25]:
import time
time_start = time.time()

import optuna
import math
import torch
import torch.nn as nn
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from tqdm import tqdm 
from torch import optim
from scipy import interpolate
from sklearn import linear_model 
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data,Dataset,DataLoader

import pytz
import datetime
nowTime = datetime.datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%Y_%m_%d_%H_%M_%S')  # Current time

# Alternative approach
torch.set_default_dtype(torch.float32)

#torch.set_default_tensor_type(torch.DoubleTensor)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [26]:
import random
from torch.backends import cudnn

def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    # Fully reproduce, but training efficiency is low
    # torch.backends.cudnn.benchmark = False
    # torch.backends.cudnn.deterministic = True

In [27]:
# read data and slicing to generate training samples
def read_data(prob_No, window_size, stride, MaxRUL):
    '''
    :param prob_No: 1,2,3,4, ----representing FD001 ~ FD004
    :param window_size: sliding window size
    :param stride: slide step length
    :param MaxRUL: maximum remaining useful life
    :return: training & testing set data (dims: num x lens/or window_size x channels) & labels (dims: num x lens/or window_size) or (dims: num x lens/1)
    '''
    min_max_scaler = MinMaxScaler()  # Normalize to [0,1] default axis=0 column
    # Use axis = 0 to represent method execution along each column or row label/index value
    # Use axis = 1 value to indicate the execution of the corresponding method along each row or column label mode
    train_load = np.loadtxt('./CMaps/train_FD00' + str(prob_No) + '.txt')
    test_load = np.loadtxt('./CMaps/test_FD00' + str(prob_No) + '.txt')
    rul_load = np.loadtxt('./CMaps/RUL_FD00' + str(prob_No) + '.txt')
    '''
    Engine number, running time, three operation modes, 21 sensors
    '''
    train_load[:, 2:] = min_max_scaler.fit_transform(train_load[:, 2:])  # Normalize (from three operation modes and 21 sensors)
    test_load[:, 2:] = min_max_scaler.transform(test_load[:, 2:])
    '''
    Engine number, running time, (three operation modes, 21 sensors) range [0-1]
    '''
    train_data = np.delete(train_load, [2, 3, 4, 5, 9, 10, 14, 20, 22, 23], axis=1)  # select sensor
    test_data = np.delete(test_load, [2, 3, 4, 5, 9, 10, 14, 20, 22, 23], axis=1)

    x = np.array(list(range(0, window_size, stride)))  # x represents time step list for time step regression
    # range (start, stop, step), create array
    # Training set
    train_data_total = []
    train_label_total = []
    unitnum1 = int(np.max(train_data[:, 0]))  # Maximum value in the first column, representing the number of training sets, 260
    LRmodel = linear_model.LinearRegression()  # Linear regression
    train_cyclen_list = []
    train_sample_num = 0

    for unit in tqdm(range(1, unitnum1 + 1), desc='Training Data Sampling FD00' + str(prob_No), ascii=False, ncols=100,
                     total=unitnum1):# One for loop, sliding window slicing 35x14, dividing a set of engine running cycle data, tqdm progress bar, traverse all engine numbers
        ind = np.where(train_data[:, 0] == unit)# Is the engine number 1? ... 2.3.4..260
        ind = ind[0]
        data = train_data[ind, 2:]# 14 sensor parameter values of engine number 1 are passed into data
        cyclens = data.shape[0]# 0 indicates the number of matrix rows, 1 indicates the number of matrix columns, indicating the total life length of the engine

        labels = [(cyclens - t if cyclens - t <= MaxRUL else MaxRUL) for t in range(1, cyclens + 1)]# Labels sliding window processing data

        if cyclens >= window_size:# Data sliding window processing
            train_cyclen_list.append(cyclens - window_size + 1)# Number of data fragments in an engine
            for i in range(0, cyclens - window_size + 1, stride):
                scaled_data = data[i:i + window_size]
                # 0-35  1-36 。。。
                train_data_total.append(scaled_data)# Collection of data slices, 30,30 per group
                train_label_total.append(torch.tensor(labels[i + window_size - 1]))

                train_sample_num += 1# Number of data fragments in an engine (data produced by sliding window of a model), accumulated to 260 engine data fragments and

    # Test set
    test_data_total = []
    test_label_total = []
    unitnum2 = int(np.max(test_data[:, 0]))
    test_cyclen_list = []
    test_sample_num = 0

    test_label = rul_load

    for unit in tqdm(range(1, unitnum2 + 1), desc='Testing Data Sampling FD00' + str(prob_No), ascii=False, ncols=100,
                     total=unitnum2):
        ind = np.where(test_data[:, 0] == unit)
        ind = ind[0]
        data = test_data[ind, 2:]
        cyclens = data.shape[0]

        if cyclens >= window_size:
            test_cyclen_list.append(cyclens - window_size + 1)
            labels = [((test_label[unit - 1] + cyclens - 1 - t)
                           if (test_label[unit - 1] + cyclens - 1 - t) <= MaxRUL else MaxRUL) for t in range(cyclens)]

            for i in range(0, cyclens - window_size + 1, stride):
                scaled_data = data[i:i + window_size]

                test_data_total.append(scaled_data)
                test_label_total.append(torch.tensor(labels[i + window_size - 1]))

                test_sample_num += 1

    return train_data_total, train_label_total, train_sample_num, train_cyclen_list, test_data_total, test_label_total, test_sample_num, test_cyclen_list

In [28]:
edge_index =[
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 13, 13, 13],
    [1, 2, 4, 5, 6, 8, 9, 10, 11, 0, 2, 4, 5, 6, 8, 9, 10, 11, 0, 1, 4, 5, 6, 8, 9, 10, 11, 7, 12, 13, 0, 1, 2, 6, 8, 10, 11, 0, 1, 2, 6, 9, 10, 11, 0, 1, 2, 4, 5, 8, 9, 10, 11, 3, 12, 13, 0, 1, 2, 4, 6, 10, 11, 0, 1, 2, 5, 6, 10, 11, 0, 1, 2, 4, 5, 6, 8, 9, 11, 0, 1, 2, 4, 5, 6, 8, 9, 10, 3, 7, 13, 3, 7, 12]
]

In [29]:
def RULdataset(data, label, edge_index):
    data_list = []
    for i in tqdm(range(len(data))):
        x = data[i].T
        node_features = torch.tensor(x, dtype=torch.float)#float
        graph_label = torch.tensor([label[i]], dtype=torch.float)#float
        edge = torch.tensor(edge_index, dtype=torch.long)#long
        data1 = Data(x=node_features, y=graph_label, edge_index=edge)
        data_list.append(data1)

    return data_list

In [30]:
#Evaluation Metrics
def RUL_Score(y_true, y_pre):

    y_true = y_true.view(-1).detach().numpy()
    y_pre = y_pre.view(-1).detach().numpy()
    d = y_pre - y_true
    mse = np.mean(np.square(d))
    Score = np.sum(np.exp(-d[d < 0] / 13) - 1) + np.sum(np.exp(d[d >= 0] / 10) - 1)

    return mse, Score

In [31]:
# Self attention mechanism

from math import sqrt

import torch.nn.functional as F

class Self_attn(nn.Module):
    def __init__(self, dim_q, dim_k, dim_v):
        super(Self_attn, self).__init__()
        self.dim_q = dim_q
        self.dim_k = dim_k
        self.dim_v = dim_v

        # Define linear transformation functions
        self.linear_q = nn.Linear(dim_q, dim_k, bias=False)
        self.linear_k = nn.Linear(dim_q, dim_k, bias=False)
        self.linear_v = nn.Linear(dim_q, dim_v, bias=False)
        self._norm_fact = 1 / sqrt(dim_k)

    def forward(self, x):
        # x: batch, n, dim_q
        # Obtain corresponding dimensions based on the text

        q = self.linear_q(x)  # batch, n, dim_k
        k = self.linear_k(x)  # batch, n, dim_k
        v = self.linear_v(x)  # batch, n, dim_v
        
        # Compute q*k's transpose and multiply by the square root of dk
        dist = torch.bmm(q, k.transpose(1, 2)) * self._norm_fact  # batch, n, n
        
        # Normalize to obtain attention coefficients
        dist = torch.softmax(dist, dim=-1)  # batch, n, n
        
        # Multiply attention coefficients by v to obtain the final scores
        att = torch.bmm(dist, v)
        return att

In [32]:
from torch.nn.utils import weight_norm

# After convolution, due to padding, the size of the new data after convolution (B) may be greater than the size of the input data (A),
# so only the first A data in the output data are retained.
# Modify the data size
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x): 
        return x[:, :, :-self.chomp_size].contiguous()


# Residual Block --- TemporalBlock
class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size1,  stride, dilation, dropout):
        super(TemporalBlock, self).__init__()
        # Scale 1
        padding1 = (kernel_size1-1) * dilation
        self.conv11 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size1,
                                            stride=stride, padding=padding1, dilation=dilation))
        self.chomp11 = Chomp1d(padding1)
        self.relu11 = nn.ReLU()
        self.dropout11 = nn.Dropout(dropout)
        
        self.conv21 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size1,
                                            stride=stride, padding=padding1, dilation=dilation))
        self.chomp21 = Chomp1d(padding1)
        self.relu21 = nn.ReLU()
        self.dropout21 = nn.Dropout(dropout)
        
        # Structure: Dilated causal convolution → Modify size → ReLU → Dropout → 
        #           Dilated causal convolution → Modify size → ReLU → Dropout 
        self.net1 = nn.Sequential(self.conv11, self.chomp11, self.relu11, self.dropout11,
                                  self.conv21, self.chomp21, self.relu21, self.dropout21)

        # Residual connection: 1x1 convolution across layers
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    # Initialize weights
    def init_weights(self):
        self.conv11.weight.data.normal_(0, 0.01)
        self.conv21.weight.data.normal_(0, 0.01)
        
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net1(x)
        res = x if self.downsample is None else self.downsample(x)  # Residual connection
        o = self.relu(out + res)
        return o


# Temporal Convolutional Network Main Structure - Overall Network
class TemporalConvNet(nn.Module):
    # num_inputs: Number of input data channels
    # num_channels: Input-output channels of each hidden layer and output layer in the network structure
    def __init__(self, num_inputs, num_channels, kernel_size, dropout):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)  # Number of network layers
        for i in range(num_levels):     # Derive the structure of each layer
            dilation_size = 2 ** i      # Dilation convolution factor is 2 to the power of i

            # The number of input channels of the first layer (input layer) is determined based on the number of original data channels,
            # otherwise, for other layers, it reads from the set network structure -- i.e., the input and output numbers of the hidden and output layers need to be set
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]

            # Each layer is a TemporalBlock with different dilation factors but the same convolutional kernel
            # padding = (kernel_size - 1) * dilation
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)   # Overall network composed of these layers connected together

    def forward(self, x):
        return self.network(x)

In [33]:
from torch_geometric.nn import GATConv, TransformerConv, BatchNorm # noqa
from torch_geometric.nn import TopKPooling, EdgePooling, ASAPooling, SAGPooling, global_mean_pool

class Model(torch.nn.Module):
    def __init__(self, window_size, num_inputs, num_channels, kernel_size, dropout, outs, head, dp, dp2):
        super(Model, self).__init__()
        self.window_size = window_size
        self.num_inputs = num_inputs

        # Temporal Convolutional Network (TCN) layer
        self.tcn = TemporalConvNet(num_inputs, num_channels, kernel_size, dropout)
        
        # Self-attention layer
        self.attn = Self_attn(window_size, window_size, window_size)
        self.relu = nn.ReLU()
        
        # Fully connected layers
        self.fc1 = nn.Linear(num_channels[-1] * window_size, 256)
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(256, 1)
        self.relu4 = nn.ReLU()

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # Reshape input for TCN
        xtcn = x.view(-1, 14, self.window_size)
        
        # TCN layer
        xtcn = self.tcn(xtcn)
        
        # Self-attention layer
        xtcn = self.attn(xtcn)
        xtcn = self.relu(xtcn)
        x = torch.flatten(xtcn, start_dim=-2, end_dim=-1)
        
        # Fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        
        x = self.fc2(x)
        x = self.relu4(x)
        
        return x

In [34]:
def train(model, mse_loss, optimizer, lossdata_train, train_loader):
    model.train()  # Activate training mode
    loss_total = []
    for data in train_loader:
        data = data.to(device)
        out = model(data)  # Perform a single forward pass
        out = torch.squeeze(out)  # Remove dimensions of size 1
        loss = torch.sqrt(mse_loss(out, data.y))  # Compute the RMSE loss
        loss_total.append(loss.item())
        loss.backward()  # Compute gradients
        optimizer.step()  # Update parameters
        optimizer.zero_grad()  # Clear gradients
    lossdata_train.append(np.mean(loss_total))
    print('Training RMSE: {}'.format(np.mean(loss_total)))
    return lossdata_train

def test(model, mse_loss, lossdata_test, test_loader):
    model.eval()  # Set model to evaluation mode
    lossi = []
    for data in test_loader:
        data = data.to(device)
        out = model(data)  # Perform a single forward pass
        out = torch.squeeze(out)  # Remove dimensions of size 1
        loss = torch.sqrt(mse_loss(out, data.y))  # Compute the RMSE loss
        lossi.append(loss.item())
    lossdata_test.append(np.mean(lossi))
    print('Testing RMSE:{}'.format(np.mean(lossi)))
    return lossdata_test

In [35]:
def eval_model(model, test_sample_num, test_cyclen_list, val_loader, Draw):
    model.eval()
    pred_rul = []
    true_rul = []
    lossi = 0
    Score = 0
    accy = 0
    
    for data in val_loader:
        data = data.to(device)
        out = model(data)
        out = torch.squeeze(out)
        
        pred_rul.append(out.item())
        true_rul.append(data.y.item())
        
        h = out.item() - data.y.item()
        lossi += h**2
        si = (math.exp((-h/13.0)) - 1) if h < 0 else (math.exp((h/10.0)) - 1)
        Score += si
        
        if -13 <= h <= 10:
            accy += 1 
    
    print('Testing RMSE: {}'.format(np.sqrt(lossi / test_sample_num)))
    print('Testing Score: {}'.format(Score))
    print('Testing Accuracy: {}'.format(accy / test_sample_num))
    
    Score = 0
    accy = 0
    loss2 = 0
    result_true = []
    result_pred = []
    
    for No_sample in range(len(test_cyclen_list)):
        y_hat = pred_rul[sum(test_cyclen_list[:No_sample+1]) - 1]
        y_true = true_rul[sum(test_cyclen_list[:No_sample+1]) - 1]
        
        result_true.append(y_true)
        result_pred.append(y_hat)
        
        h = y_hat - y_true
        loss2 = loss2 + (h**2)
        si = (math.e**(-h/13.0) - 1) if h < 0 else (math.e**(h/10.0) - 1)
        Score += si
        
        if -13 <= h <= 10:
            accy += 1
            
    RMSE = np.sqrt(loss2 / len(test_cyclen_list))
    print('Testing RMSE for the end of samples: {}'.format(RMSE))
    print('Testing Score for the end of samples: {}'.format(Score))
    print('Testing Accuracy for the end of samples: {}'.format(accy / len(test_cyclen_list)))
    
    return RMSE

In [36]:
prob_No = 1
window_size = 50

print('Window size: {}'.format(window_size))
nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')

print(str(nowTime) + ': Initialization completed. Reading data, please wait...')

train_data, train_label, train_sample_num, train_cyclen_list, test_data, test_label, test_sample_num, test_cyclen_list = read_data(prob_No, window_size, 1, 125)

nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
print(str(nowTime) + ': Data reading completed.')
print(len(train_data), train_sample_num, test_sample_num)

batch_size = 256
shuffle = True

train_dataset = RULdataset(data=train_data, label=train_label, edge_index=edge_index)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

test_dataset = RULdataset(data=test_data, label=test_label, edge_index=edge_index)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

val_dataset = RULdataset(data=test_data, label=test_label, edge_index=edge_index)
val_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

Window size: 50
2025_01_19_21_54_36: Initialization completed. Reading data, please wait...


Testing Data Sampling FD001: 100%|██████████████████████████████| 100/100 [00:00<00:00, 1651.87it/s]


2025_01_19_21_54_38: Data reading completed.
15731 15731 8255


100%|██████████| 15731/15731 [00:01<00:00, 15332.85it/s]
c:\Users\bouch\AppData\Local\Programs\Python\Python39\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 8255/8255 [00:00<00:00, 27428.78it/s]


In [37]:
train_loader

In [38]:
def main(trial):
    num_kernel = 16
    n_layers = 3
    level_channels = [num_kernel for layers in range(n_layers)]
    sk = 5
    drop = 0.2
    head = 3
    outs = 5
    dp = 0.2
    dp2 = 0.6
    model = Model(window_size, num_inputs=14, num_channels=level_channels, kernel_size=sk, dropout=drop, outs=outs, head=head, dp=dp, dp2=dp2)
    model.to(device)

    mse_loss = nn.MSELoss()

    lr = 0.01
    optimizer = optim.Adam(model.parameters(), lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, 40, gamma=0.1, last_epoch=-1)

    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(str(nowTime) + ': Training model starts -----------------------------------------------')

    lossdata_train = []
    lossdata_test = []
    iter_n = 120
    meidong = 0

    for epoch in range(iter_n):
        print('epoch:', epoch)
        lossdata_train = train(model, mse_loss, optimizer, lossdata_train, train_loader)
        lossdata_test = test(model, mse_loss, lossdata_test, test_loader)
        scheduler.step()

        if lossdata_train[-1] > 10000 or lossdata_test[-1] > 10000:
            break
        elif lossdata_train[-1] > 80 or lossdata_test[-1] > 100:
            meidong += 1
            if meidong > 10:
                break 

    if lossdata_train[-1] > 10000 or lossdata_test[-1] > 10000:
        print('Training failed due to encountering inf.')
    elif meidong > 10:
        print('Training failed due to lack of loss reduction.')
    else:
        plt.title("Loss with Epoch")
        plt.xlabel("Epoch")
        plt.xlim(0, int(iter_n))
        plt.ylabel("RMSE Loss")

        plt.plot([i for i in range(iter_n)], lossdata_train, 'r', label='Training set')
        plt.plot([i for i in range(iter_n)], lossdata_test, 'b', label='Testing set')
        plt.legend()
        plt.show()
    
    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    save_dir = r"F:\ros2\phm_model"
    torch.save(model.state_dict(), os.path.join(save_dir, "model.pth"))

    print(str(nowTime) + ': Training completed. Model saved as DS-STFN_trained_' + str(iter_n) + 'iter_' + str(nowTime) + '.pth')

    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(str(nowTime) + ': Evaluating the final model')
    RMSE = eval_model(model, test_sample_num, test_cyclen_list, val_loader, Draw=True)
    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(str(nowTime) + ': Experiment evaluation completed')
    print('-----------------------------------------------')
    return RMSE

In [39]:
data_batch = next(iter(train_loader)).to(device)
data_batch

DataBatch(x=[3584, 50], edge_index=[2, 24064], y=[256], batch=[3584], ptr=[257])

In [40]:
from torchviz import make_dot

# Load a single batch from the train loader
data_batch = next(iter(train_loader)).to(device)

# Forward pass with the model
model_output = model(data_batch)

# Visualize the computation graph
dot = make_dot(model_output, params=dict(model.named_parameters()), show_attrs=True, show_saved=True)
dot.render("model_architecture", format="png")


DataBatch(x=[3584, 50], edge_index=[2, 24064], y=[256], batch=[3584], ptr=[257])


In [41]:
data_batch = next(iter(train_loader))
print(data_batch)


NameError: name 'model' is not defined

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [65]:
import os
import optuna

# Ensure the directory exists
os.makedirs(r"F:\ros2\phm_model", exist_ok=True)

# Define the objective function
def objective(trial):
    RMSE = main(trial)  # Ensure main(trial) is defined and returns RMSE
    return RMSE

# Create and optimize the study
study = optuna.create_study(
    storage=r'sqlite:///F:/ros2/phm_model/optuna_study.db',
    direction="minimize"
)
study.optimize(objective, n_trials=15, timeout=40000, show_progress_bar=False)

# Output results
print("Best Parameters:", study.best_params)
print("Best Value (RMSE):", study.best_value)


AttributeError: 'Tensor' object has no attribute 'x'

In [70]:
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

tensor([[ 78.2729],
        [ 36.2622],
        [123.5248],
        [ 29.8357],
        [ 22.5238],
        [ 13.7712],
        [ 39.1777],
        [ 82.6147],
        [103.9785],
        [ 39.3488],
        [114.0385],
        [ 84.2049],
        [  7.1783],
        [115.3101],
        [122.1537],
        [ 87.7092],
        [ 78.7403],
        [119.2342],
        [125.7785],
        [  4.8667],
        [113.0875],
        [108.8999],
        [119.5241],
        [ 33.5120],
        [ 47.5251],
        [ 65.5653],
        [131.9938],
        [123.1424],
        [108.8336],
        [111.8369],
        [ 99.4475],
        [ 14.7517],
        [118.7794],
        [110.3830],
        [ 98.9049],
        [ 65.0915],
        [126.9383],
        [  6.8929],
        [ 43.5221],
        [ 30.7508],
        [116.3535],
        [ 94.1596],
        [ 55.2111],
        [105.4020],
        [128.1735],
        [119.2595],
        [ 45.6833],
        [  9.3569],
        [  3.2505],
        [126.9072],


In [54]:
!pip install -q --no-cache-dir torch==2.1.2+cu121 --extra-index-url https://download.pytorch.org/whl/cu121
!pip install -q --no-cache-dir torch-geometric

In [52]:
model


Model(
  (tcn): TemporalConvNet(
    (network): Sequential(
      (0): TemporalBlock(
        (conv11): Conv1d(14, 16, kernel_size=(5,), stride=(1,), padding=(4,))
        (chomp11): Chomp1d()
        (relu11): ReLU()
        (dropout11): Dropout(p=0.2, inplace=False)
        (conv21): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(4,))
        (chomp21): Chomp1d()
        (relu21): ReLU()
        (dropout21): Dropout(p=0.2, inplace=False)
        (net1): Sequential(
          (0): Conv1d(14, 16, kernel_size=(5,), stride=(1,), padding=(4,))
          (1): Chomp1d()
          (2): ReLU()
          (3): Dropout(p=0.2, inplace=False)
          (4): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(4,))
          (5): Chomp1d()
          (6): ReLU()
          (7): Dropout(p=0.2, inplace=False)
        )
        (downsample): Conv1d(14, 16, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
      )
      (1): TemporalBlock(
        (conv11): Conv1d(16, 16, kernel_size=(5,), 

In [19]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,state
0,0,12.715796,2024-05-24 21:00:16.392641,2024-05-24 21:04:06.864579,0 days 00:03:50.471938,COMPLETE
1,1,84.226906,2024-05-24 21:04:06.867446,2024-05-24 21:04:53.835828,0 days 00:00:46.968382,COMPLETE
2,2,12.711702,2024-05-24 21:04:53.838399,2024-05-24 21:08:43.211317,0 days 00:03:49.372918,COMPLETE
3,3,84.226906,2024-05-24 21:08:43.215053,2024-05-24 21:09:30.193899,0 days 00:00:46.978846,COMPLETE
4,4,13.251575,2024-05-24 21:09:30.196835,2024-05-24 21:13:19.256990,0 days 00:03:49.060155,COMPLETE
5,5,84.226906,2024-05-24 21:13:19.260032,2024-05-24 21:14:06.178619,0 days 00:00:46.918587,COMPLETE
6,6,12.127952,2024-05-24 21:14:06.181207,2024-05-24 21:17:55.211409,0 days 00:03:49.030202,COMPLETE
7,7,12.336531,2024-05-24 21:17:55.214745,2024-05-24 21:21:44.983707,0 days 00:03:49.768962,COMPLETE
8,8,84.226906,2024-05-24 21:21:44.986383,2024-05-24 21:22:32.081477,0 days 00:00:47.095094,COMPLETE
9,9,12.054134,2024-05-24 21:22:32.084484,2024-05-24 21:26:21.645594,0 days 00:03:49.561110,COMPLETE


In [20]:
#Iteration Results Over Multiple Trials
optuna.visualization.plot_optimization_history(study)

In [21]:
#Parameter Importance
#optuna.visualization.plot_param_importances(study)

In [22]:
optuna.visualization.plot_contour(study)

In [23]:
optuna.visualization.plot_parallel_coordinate(study)

In [1]:
import sys
print("python", sys.version)
import torch
print("torch", torch.__version__)
print("cuda", torch.version.cuda)
print(torch.cuda.is_available())

import time
time_start = time.time()

import os
import optuna
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch import optim
from scipy import interpolate
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import pytz
import datetime
nowTime = datetime.datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%Y_%m_%d_%H_%M_%S')

# PyTorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# For reproducibility
import random
from torch.backends import cudnn

def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

python 3.9.12 (tags/v3.9.12:b28265d, Mar 23 2022, 23:52:46) [MSC v.1929 64 bit (AMD64)]
torch 2.1.2+cu121
cuda 12.1
True


In [2]:
###############################################################################
#                         1) Read & Pre-Process Data                          #
###############################################################################
def read_data(prob_No, window_size, stride, MaxRUL):
    """
    Reads CMAPSS data, applies min-max normalization, selects specific sensors,
    and slices sequences of length window_size (with given stride).  
    Returns train_data, train_label, test_data, test_label, etc.
    """
    min_max_scaler = MinMaxScaler()
    
    # Load original FD00X data
    train_load = np.loadtxt('./CMaps/train_FD00' + str(prob_No) + '.txt')
    test_load = np.loadtxt('./CMaps/test_FD00' + str(prob_No) + '.txt')
    rul_load = np.loadtxt('./CMaps/RUL_FD00' + str(prob_No) + '.txt')
    
    # Normalize from columns 2 onward (3 op-settings + 21 sensors => 24 columns)
    train_load[:, 2:] = min_max_scaler.fit_transform(train_load[:, 2:])
    test_load[:, 2:]  = min_max_scaler.transform(test_load[:, 2:])
    
    # Now select 14 sensors from those 21
    # (You delete columns [2,3,4,5,9,10,14,20,22,23] from the entire array => effectively dropping certain sensors.)
    train_data = np.delete(train_load, [2, 3, 4, 5, 9, 10, 14, 20, 22, 23], axis=1)
    test_data  = np.delete(test_load,  [2, 3, 4, 5, 9, 10, 14, 20, 22, 23], axis=1)

    # Prepare container
    train_data_total = []
    train_label_total = []
    test_data_total = []
    test_label_total = []

    # Number of engines in training
    unitnum1 = int(np.max(train_data[:, 0]))  # e.g. FD001 => 100 engines
    train_cyclen_list = []
    train_sample_num  = 0

    # Build training slices
    for unit in tqdm(range(1, unitnum1 + 1), desc='Train Sampling FD00' + str(prob_No)):
        ind = np.where(train_data[:, 0] == unit)[0]
        data_unit = train_data[ind, 2:]  # shape => [cycles, 14]
        cyclens = data_unit.shape[0]

        # RUL label for each cycle
        labels = [(cyclens - t if (cyclens - t) <= MaxRUL else MaxRUL) for t in range(1, cyclens + 1)]

        if cyclens >= window_size:
            train_cyclen_list.append(cyclens - window_size + 1)
            for i in range(0, cyclens - window_size + 1, stride):
                # slice [i : i+window_size, :] => shape [window_size, 14]
                data_slice = data_unit[i : i + window_size]
                lbl = labels[i + window_size - 1]  # RUL label at last point in the window
                
                train_data_total.append(data_slice)                # shape [50,14]
                train_label_total.append(torch.tensor(lbl))        # single number
                train_sample_num += 1

    # Number of engines in testing
    unitnum2 = int(np.max(test_data[:, 0]))
    test_cyclen_list = []
    test_sample_num  = 0

    # RUL for each engine
    test_label_full = rul_load  # shape => [#engines]

    # Build testing slices
    for unit in tqdm(range(1, unitnum2 + 1), desc='Test Sampling FD00' + str(prob_No)):
        ind = np.where(test_data[:, 0] == unit)[0]
        data_unit = test_data[ind, 2:]  # shape => [cycles, 14]
        cyclens = data_unit.shape[0]

        if cyclens >= window_size:
            test_cyclen_list.append(cyclens - window_size + 1)
            # build RUL for each cycle in this unit
            labels = []
            for t in range(cyclens):
                # test_label_full[unit - 1] => RUL offset for this engine
                tmp_rul = test_label_full[unit - 1] + (cyclens - 1 - t)
                tmp_rul = tmp_rul if tmp_rul <= MaxRUL else MaxRUL
                labels.append(tmp_rul)

            for i in range(0, cyclens - window_size + 1, stride):
                data_slice = data_unit[i : i + window_size]
                lbl = labels[i + window_size - 1]
                test_data_total.append(data_slice)
                test_label_total.append(torch.tensor(lbl))
                test_sample_num += 1

    return (train_data_total, train_label_total, train_sample_num, train_cyclen_list,
            test_data_total,  test_label_total,  test_sample_num,  test_cyclen_list)

In [3]:

###############################################################################
#                              2) Define Model                                #
###############################################################################
from math import sqrt

class Self_attn(nn.Module):
    def __init__(self, dim_q, dim_k, dim_v):
        super(Self_attn, self).__init__()
        self.dim_q = dim_q
        self.dim_k = dim_k
        self.dim_v = dim_v

        self.linear_q = nn.Linear(dim_q, dim_k, bias=False)
        self.linear_k = nn.Linear(dim_q, dim_k, bias=False)
        self.linear_v = nn.Linear(dim_q, dim_v, bias=False)
        self._norm_fact = 1 / math.sqrt(dim_k)

    def forward(self, x):
        # x => [B, channels, seq_len]  (?), but we do scaled dot-product attention along dimension #2
        # Actually, in your code, it was used as [B, n, dim_q].
        # We'll assume x is [B, channels, seq_len].
        q = self.linear_q(x)  # => [B, channels, dim_k]
        k = self.linear_k(x)  # => [B, channels, dim_k]
        v = self.linear_v(x)  # => [B, channels, dim_v]

        # batch matrix multiply
        dist = torch.bmm(q, k.transpose(1, 2)) * self._norm_fact  # => [B, channels, channels]
        dist = torch.softmax(dist, dim=-1)                        # => [B, channels, channels]
        att  = torch.bmm(dist, v)                                 # => [B, channels, dim_v]
        return att

from torch.nn.utils import weight_norm

class Chomp1d(nn.Module):
    """Truncates extra padding after 1D convolutions if needed."""
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size1, stride, dilation, dropout):
        super(TemporalBlock, self).__init__()
        padding1 = (kernel_size1 - 1) * dilation
        self.conv11 = weight_norm(nn.Conv1d(
            n_inputs, n_outputs, kernel_size1,
            stride=stride, padding=padding1, dilation=dilation
        ))
        self.chomp11 = Chomp1d(padding1)
        self.relu11  = nn.ReLU()
        self.dropout11 = nn.Dropout(dropout)
        
        self.conv21 = weight_norm(nn.Conv1d(
            n_outputs, n_outputs, kernel_size1,
            stride=stride, padding=padding1, dilation=dilation
        ))
        self.chomp21 = Chomp1d(padding1)
        self.relu21  = nn.ReLU()
        self.dropout21 = nn.Dropout(dropout)
        
        self.net1 = nn.Sequential(
            self.conv11, self.chomp11, self.relu11, self.dropout11,
            self.conv21, self.chomp21, self.relu21, self.dropout21
        )

        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if (n_inputs != n_outputs) else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv11.weight.data.normal_(0, 0.01)
        self.conv21.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net1(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers.append(
                TemporalBlock(in_channels, out_channels, kernel_size,
                              stride=1, dilation=dilation_size, dropout=dropout)
            )
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # x => [B, C, T]
        return self.network(x)
###############################################################################
#                     The Main Model (no PyG, single Tensor)                  #
###############################################################################
class Model(nn.Module):
    def __init__(self, window_size, num_inputs, num_channels, kernel_size,
                 dropout, outs, head, dp, dp2):
        super(Model, self).__init__()
        self.window_size = window_size
        self.num_inputs  = num_inputs

        # TCN
        self.tcn = TemporalConvNet(num_inputs, num_channels, kernel_size, dropout)

        # Self-attention
        self.attn = Self_attn(dim_q=window_size, dim_k=window_size, dim_v=window_size)
        self.relu = nn.ReLU()

        # FC layers
        self.fc1  = nn.Linear(num_channels[-1] * window_size, 256)
        self.relu3= nn.ReLU()
        self.fc2  = nn.Linear(256, 1)
        self.relu4= nn.ReLU()

    def forward(self, x):
        """
        x: shape [B, 14, window_size]
        """
        # 1) TCN (expects [B, channels, length])
        xtcn = self.tcn(x)  # => [B, num_channels[-1], window_size]

        # 2) Self-attention => shape still [B, num_channels[-1], window_size]
        xtcn = self.attn(xtcn)
        xtcn = self.relu(xtcn)

        # 3) Flatten last two dims => [B, num_channels[-1]*window_size]
        x_flat = torch.flatten(xtcn, start_dim=1)

        # 4) FC
        x_fc = self.fc1(x_flat)
        x_fc = self.relu3(x_fc)
        out  = self.fc2(x_fc)
        out  = self.relu4(out)
        return out  # => [B, 1]


In [68]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm

###############################################################################
#                           Self-Attention Revised                            #
###############################################################################
class Self_attn(nn.Module):
    def __init__(self, dim_q, dim_k, dim_v):
        """
        dim_q, dim_k, dim_v correspond to the *channel* dimension,
        i.e. out_channels from the TCN.

        We'll do self-attention *across the time dimension*.
        """
        super(Self_attn, self).__init__()
        self.dim_q = dim_q
        self.dim_k = dim_k
        self.dim_v = dim_v

        self.linear_q = nn.Linear(dim_q, dim_k, bias=False)
        self.linear_k = nn.Linear(dim_q, dim_k, bias=False)
        self.linear_v = nn.Linear(dim_q, dim_v, bias=False)
        self._norm_fact = 1 / math.sqrt(dim_k)

    def forward(self, x):
        """
        x => [B, channels=dim_q, time=T]

        We'll transpose to [B, time=T, channels=dim_q] before applying linear.
        Then do scaled dot-product across time dimension.
        """
        B, C, T = x.shape  # e.g. [B, 16, 50]
        # Transpose => [B, T, C]
        x_t = x.transpose(1, 2)  # shape [B, T, C=dim_q]

        q = self.linear_q(x_t)  # => [B, T, dim_k]
        k = self.linear_k(x_t)  # => [B, T, dim_k]
        v = self.linear_v(x_t)  # => [B, T, dim_v]

        # Scaled dot-product attention => dist shape [B, T, T]
        dist = torch.bmm(q, k.transpose(1, 2)) * self._norm_fact
        dist = torch.softmax(dist, dim=-1)

        # Multiply by v => shape [B, T, dim_v]
        att = torch.bmm(dist, v)

        # Transpose back to [B, dim_v, T]
        att = att.transpose(1, 2)  # => [B, dim_v, T]
        return att

###############################################################################
#                               TCN Building Blocks                           #
###############################################################################
class Chomp1d(nn.Module):
    """Truncates extra padding after 1D convolutions if needed."""
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()

class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size1, stride, dilation, dropout):
        super(TemporalBlock, self).__init__()
        padding1 = (kernel_size1 - 1) * dilation
        self.conv11 = weight_norm(nn.Conv1d(
            n_inputs, n_outputs, kernel_size1,
            stride=stride, padding=padding1, dilation=dilation
        ))
        self.chomp11 = Chomp1d(padding1)
        self.relu11  = nn.ReLU()
        self.dropout11 = nn.Dropout(dropout)
        
        self.conv21 = weight_norm(nn.Conv1d(
            n_outputs, n_outputs, kernel_size1,
            stride=stride, padding=padding1, dilation=dilation
        ))
        self.chomp21 = Chomp1d(padding1)
        self.relu21  = nn.ReLU()
        self.dropout21 = nn.Dropout(dropout)
        
        self.net1 = nn.Sequential(
            self.conv11, self.chomp11, self.relu11, self.dropout11,
            self.conv21, self.chomp21, self.relu21, self.dropout21
        )

        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if (n_inputs != n_outputs) else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv11.weight.data.normal_(0, 0.01)
        self.conv21.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net1(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout):
        """
        num_inputs = in_channels (e.g. 14 sensors)
        num_channels = list of out_channels for each TCN layer, e.g. [16,16,16]
        kernel_size = TCN kernel, e.g. 5
        dropout = e.g. 0.2
        """
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers.append(
                TemporalBlock(in_channels, out_channels, kernel_size,
                              stride=1, dilation=dilation_size, dropout=dropout)
            )
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        # x => [B, in_channels, T], e.g. [B, 14, window_size]
        return self.network(x)

###############################################################################
#                           The Main Model (Single Tensor)                    #
###############################################################################
class Model(nn.Module):
    def __init__(self, window_size, num_inputs, num_channels, kernel_size,
                 dropout, outs, head, dp, dp2):
        super(Model, self).__init__()
        self.window_size = window_size
        self.num_inputs  = num_inputs

        # 1) TCN
        #    Expects input shape [B, num_inputs=14, T=window_size]
        #    Returns [B, num_channels[-1], T]
        self.tcn = TemporalConvNet(num_inputs, num_channels, kernel_size, dropout)

        # 2) Self-attention
        #    We'll do attention across time dimension T, so the "embedding" dimension is num_channels[-1].
        #    e.g. if num_channels[-1] = 16, we set dim_q = dim_k = dim_v = 16.
        out_channels = num_channels[-1]
        self.attn = Self_attn(dim_q=out_channels, dim_k=out_channels, dim_v=out_channels)
        self.relu = nn.ReLU()

        # 3) Fully-Connected Layers
        #    Flatten => [B, out_channels*T], so in_features = out_channels * window_size
        self.fc1  = nn.Linear(out_channels * window_size, 256)
        self.relu3= nn.ReLU()
        self.fc2  = nn.Linear(256, 1)
        self.relu4= nn.ReLU()

    def forward(self, x):
        """
        x: shape [B, 14, window_size]
        1) TCN => [B, out_channels, window_size]
        2) Self-attn => [B, out_channels, window_size] (same shape)
        3) Flatten => [B, out_channels*window_size]
        4) FC => [B, 1]
        """
        # Step 1: TCN
        xtcn = self.tcn(x)  # => [B, num_channels[-1], window_size]

        # Step 2: Self-attn
        xtcn = self.attn(xtcn)  # => [B, num_channels[-1], window_size]
        xtcn = self.relu(xtcn)

        # Step 3: Flatten => [B, num_channels[-1]*window_size]
        x_flat = torch.flatten(xtcn, start_dim=1)

        # Step 4: FC
        x_fc = self.fc1(x_flat)
        x_fc = self.relu3(x_fc)
        out  = self.fc2(x_fc)
        out  = self.relu4(out)
        return out  # => [B, 1]


In [69]:

###############################################################################
#                       3) Build a Standard Dataset/Loader                    #
###############################################################################
class RULStandardDataset(Dataset):
    def __init__(self, data_list, label_list):
        """
        data_list: List of np arrays => each [window_size, 14]
        label_list: List/Tensor => each a single scalar
        We will convert them to a single 3D array [N, 14, window_size].
        """
        # Convert to shape [N, window_size, 14]
        data_arr = np.array(data_list, dtype=np.float32)  # => [N, window_size, 14]
        labels   = np.array([lbl.item() if hasattr(lbl, 'item') else lbl
                             for lbl in label_list], dtype=np.float32)  # => [N]

        # Permute to [N, 14, window_size]
        data_arr = np.transpose(data_arr, (0, 2, 1))
        self.data  = torch.tensor(data_arr, dtype=torch.float32)
        self.label = torch.tensor(labels,   dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns (x, y), where
          x => [14, window_size]
          y => scalar
        """
        return self.data[idx], self.label[idx]


In [70]:
###############################################################################
#                    4) Training, Testing, and Evaluation                     #
###############################################################################
def train_one_epoch(model, mse_loss, optimizer, loader, lossdata_train):
    model.train()
    losses = []
    for x_batch, y_batch in loader:
        x_batch = x_batch.to(device)        # [B, 14, window_size]
        y_batch = y_batch.to(device)        # [B]

        out = model(x_batch).squeeze(dim=1) # => [B]
        loss = torch.sqrt(mse_loss(out, y_batch))
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    mean_loss = np.mean(losses)
    lossdata_train.append(mean_loss)
    print(f"Training RMSE: {mean_loss:.4f}")
    return lossdata_train

def test_one_epoch(model, mse_loss, loader, lossdata_test):
    model.eval()
    losses = []
    with torch.no_grad():
        for x_batch, y_batch in loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            out = model(x_batch).squeeze(dim=1)
            loss = torch.sqrt(mse_loss(out, y_batch))
            losses.append(loss.item())

    mean_loss = np.mean(losses)
    lossdata_test.append(mean_loss)
    print(f"Testing RMSE: {mean_loss:.4f}")
    return lossdata_test

def eval_model(model, test_sample_num, test_cyclen_list, loader, Draw):
    """
    Similar logic to your original eval_model, but now we iterate over x_batch, y_batch in single-sample or small batch.
    Then we re-collect predictions in the same order they were constructed.
    """
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for x_batch, y_batch in loader:
            x_batch = x_batch.to(device)
            out = model(x_batch).squeeze(dim=1)  # shape [batch_size]
            preds.extend(out.cpu().numpy().tolist())
            trues.extend(y_batch.numpy().tolist())

    # Convert to arrays
    preds = np.array(preds)
    trues = np.array(trues)

    # Calculate overall RMSE, Score, Accuracy
    diff = preds - trues
    lossi = np.sum(diff**2)
    Score = 0
    accy  = 0
    for h in diff:
        si = (math.exp((-h/13.0)) - 1) if h < 0 else (math.exp((h/10.0)) - 1)
        Score += si
        if -13 <= h <= 10:
            accy += 1

    rmse_all = math.sqrt(lossi / test_sample_num)
    print(f"Testing RMSE (all slices): {rmse_all:.4f}")
    print(f"Testing Score: {Score:.4f}")
    print(f"Testing Accuracy: {accy/test_sample_num:.4f}")

    # Evaluate end-of-engine slices (like original code)
    Score2   = 0
    accy2    = 0
    loss2    = 0
    N_engines= len(test_cyclen_list)
    result_true = []
    result_pred = []
    cum_idx = 0
    for i, cyc_len in enumerate(test_cyclen_list):
        # The last slice for engine i is => index = (cum_idx + cyc_len - 1)
        idx_engine_last = cum_idx + cyc_len - 1
        y_hat = preds[idx_engine_last]
        y_true= trues[idx_engine_last]

        result_true.append(y_true)
        result_pred.append(y_hat)

        h = y_hat - y_true
        loss2 += (h**2)
        si = (math.e**(-h/13.0) - 1) if h < 0 else (math.e**(h/10.0) - 1)
        Score2 += si
        if -13 <= h <= 10:
            accy2 += 1

        cum_idx += cyc_len

    RMSE_end = math.sqrt(loss2 / N_engines)
    print(f"Testing RMSE for the end of samples: {RMSE_end:.4f}")
    print(f"Testing Score for the end of samples: {Score2:.4f}")
    print(f"Testing Accuracy for the end of samples: {accy2/N_engines:.4f}")

    return RMSE_end


In [71]:
###############################################################################
#                      5) Main + Hyperparameter Tuning                        #
###############################################################################
def main(trial):
    # Hyperparams (some are fixed here, some could be sampled from trial)
    num_kernel = 16
    n_layers   = 3
    level_channels = [num_kernel] * n_layers
    sk   = 5
    drop = 0.2
    head = 3
    outs = 5
    dp   = 0.2
    dp2  = 0.6

    model = Model(
        window_size=window_size,
        num_inputs=14,
        num_channels=level_channels,
        kernel_size=sk,
        dropout=drop,
        outs=outs,
        head=head,
        dp=dp,
        dp2=dp2
    ).to(device)

    mse_loss = nn.MSELoss()
    lr       = 0.01
    optimizer= optim.Adam(model.parameters(), lr=lr)
    scheduler= optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1, last_epoch=-1)

    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(f"{nowTime}: Training model starts -----------------------------------------------")

    lossdata_train = []
    lossdata_test  = []
    iter_n = 120
    meidong = 0

    for epoch in range(iter_n):
        print(f"epoch: {epoch}")
        # Train one epoch
        train_one_epoch(model, mse_loss, optimizer, train_loader, lossdata_train)
        # Test one epoch
        test_one_epoch(model, mse_loss, test_loader, lossdata_test)
        scheduler.step()

        if lossdata_train[-1] > 10000 or lossdata_test[-1] > 10000:
            print("Early break due to huge loss.")
            break
        elif lossdata_train[-1] > 80 or lossdata_test[-1] > 100:
            meidong += 1
            if meidong > 10:
                print("Training failed due to lack of loss reduction.")
                break

    # Plot
    if (lossdata_train[-1] < 10000) and (meidong <= 10):
        plt.title("Loss vs. Epoch")
        plt.xlabel("Epoch")
        plt.xlim(0, iter_n)
        plt.ylabel("RMSE Loss")
        plt.plot(range(len(lossdata_train)), lossdata_train, 'r', label='Training')
        plt.plot(range(len(lossdata_test)),  lossdata_test,  'b', label='Testing')
        plt.legend()
        plt.show()

    # Save model
    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    save_dir = r"F:\ros2\phm_model"
    os.makedirs(save_dir, exist_ok=True)
    torch.save(model.state_dict(), os.path.join(save_dir, "model.pth"))
    print(f"{nowTime}: Training completed. Model saved.")

    # Final evaluation
    print(f"{nowTime}: Evaluating final model on validation set.")
    RMSE_end = eval_model(model, test_sample_num, test_cyclen_list, val_loader, Draw=True)
    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(f"{nowTime}: Experiment evaluation completed.")
    print('-----------------------------------------------')
    return RMSE_end

In [72]:

###############################################################################
#                      6) Script Entry: Data, Datasets, etc.                  #
###############################################################################
if __name__ == "__main__":
    prob_No     = 1
    window_size = 50
    print(f"Window size: {window_size}")

    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(f"{nowTime}: Initialization completed. Reading data ...")

    (train_data, train_label, train_sample_num, train_cyclen_list,
     test_data,  test_label,  test_sample_num,  test_cyclen_list) = read_data(prob_No, window_size, 1, 125)

    nowTime = datetime.datetime.now(pytz.timezone('Asia/Dhaka')).strftime('%Y_%m_%d_%H_%M_%S')
    print(f"{nowTime}: Data reading completed.")
    print(len(train_data), train_sample_num, test_sample_num)

    batch_size = 256

    # Create standard dataset + dataloader
    train_dataset = RULStandardDataset(train_data, train_label)
    test_dataset  = RULStandardDataset(test_data,  test_label)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

    # For final evaluation with single-sample or batch=1:
    # We'll keep a "val_loader" that has batch_size=1 if you want 1-sample evaluation
    val_loader   = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # Quick check:
    print("train_loader:", train_loader)
    x_batch, y_batch = next(iter(train_loader))
    print("Sample x_batch shape:", x_batch.shape, "Sample y_batch shape:", y_batch.shape)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Optional: If you do not want to run Optuna, you can just do main(None).
    import optuna

    def objective(trial):
        RMSE = main(trial)
        return RMSE

    study = optuna.create_study(
        storage='sqlite:///F:/ros2/phm_model/optuna_study.db',
        direction="minimize"
    )
    study.optimize(objective, n_trials=15, timeout=40000, show_progress_bar=False)

    print("Best Parameters:", study.best_params)
    print("Best Value (RMSE):", study.best_value)

    pruned_trials   = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    print("Study statistics:")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial
    print("  Value: ", trial.value)
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # Example of how to do SHAP once the model is fully trained:
    #   x_batch => shape [B, 14, 50]
    #   baseline_data => shape [some_baseline_count, 14, 50]
    #   ...
    #   explainer = shap.DeepExplainer(model, baseline_data)
    #   shap_values = explainer.shap_values(x_batch)
    #   shap.summary_plot(shap_values, x_batch, ...)



Window size: 50
2025_01_28_22_03_06: Initialization completed. Reading data ...


Test Sampling FD001:  72%|███████▏  | 72/100 [00:02<00:00, 29.06it/s]


KeyboardInterrupt: 

In [73]:

# Load your model
model = Model(
    window_size=50,
    num_inputs=14,
    num_channels=[16,16,16],
    kernel_size=5,
    dropout=0.2,
    outs=5,
    head=3,
    dp=0.2,
    dp2=0.6
)

model.load_state_dict(torch.load("model.pth"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.


RuntimeError: Error(s) in loading state_dict for Model:
	size mismatch for attn.linear_q.weight: copying a param with shape torch.Size([50, 50]) from checkpoint, the shape in current model is torch.Size([16, 16]).
	size mismatch for attn.linear_k.weight: copying a param with shape torch.Size([50, 50]) from checkpoint, the shape in current model is torch.Size([16, 16]).
	size mismatch for attn.linear_v.weight: copying a param with shape torch.Size([50, 50]) from checkpoint, the shape in current model is torch.Size([16, 16]).

In [16]:
all_train_indices = list(range(len(train_dataset)))
random.shuffle(all_train_indices)
baseline_idxs = all_train_indices[:10]
baseline_subset = Subset(train_dataset, baseline_idxs)
baseline_loader = DataLoader(baseline_subset, batch_size=10, shuffle=False)

# Extract x,y from the loader => shape [10,14,50]
baseline_x, baseline_y = next(iter(baseline_loader))
baseline_x = baseline_x.to(device)  # move to GPU if needed

# For test, pick ~5 random samples from test_dataset
all_test_indices = list(range(len(test_dataset)))
random.shuffle(all_test_indices)
test_idxs = all_test_indices[:5]
test_subset = Subset(test_dataset, test_idxs)
test_loader = DataLoader(test_subset, batch_size=5, shuffle=False)

test_x, test_y = next(iter(test_loader))
test_x = test_x.to(device)  # shape [5,14,50]

print("baseline_x:", baseline_x.shape)  # => [10,14,50]
print("test_x:",     test_x.shape) 

NameError: name 'Subset' is not defined

In [33]:
explainer = shap.DeepExplainer(model, baseline_x)

# Use DeepExplainer but disable the sum check
shap_values = explainer.shap_values(test_x, check_additivity=False)


In [37]:
print("Number of feature_names:", len(feature_names))


Number of feature_names: 700


In [17]:
import torch
import shap

# 1) Suppose 'shap_values' is a NumPy array => shape [14, 50, 1] for ONE sample
shap_values_t = torch.from_numpy(shap_values)  # => torch.Size([14, 50, 1])

# 2) Suppose 'test_x' is already a PyTorch tensor => shape [14, 50] for ONE sample
test_x_t = test_x  # no from_numpy call needed

# 3) Remove the trailing output dimension => [14, 50]
shap_vals_2d = shap_values_t.squeeze(-1)    # shape => [14, 50]

# Example: shap_vals => shape [5,14,50,1], or [5,14,50]
# 1) Squeeze out any trailing dimension => [5,14,50]
sv_3d = shap_vals_2d.squeeze(-1) if shap_vals_2d.ndim==4 else shap_vals_2d  

# 2) Flatten each sample => shape [5, 700] instead of [1,3500]
B = sv_3d.shape[0]  # batch_size=5
sv_2d = sv_3d.reshape(B, 14*50)  # => [5, 700]

print("Final shap_vals shape:", sv_2d.shape) 
# => (5, 700) not (1, 3500)!

# 3) Convert to numpy for summary_plot
shap_vals_numpy = sv_2d.cpu().numpy()

# 4) Same flatten for test_x => shape [5,700]
tx_2d = test_x.reshape(B, 14*50).cpu().numpy()

# 5) Provide 700 names => 'Sensor s, Time t'
feature_names = []
for s in range(14):
    for t in range(50):
        feature_names.append(f"Sensor {s}, Time {t}")
# length(feature_names)=700

# 6) Now shap.summary_plot() sees 5x700 features & 700 labels
shap.summary_plot(
    shap_vals_numpy,
    tx_2d,
    feature_names=feature_names,
    plot_type="bar",
    max_display=20
)



NameError: name 'shap_values' is not defined

In [1]:
!pip install timeshap

   ---------------------------------------- 0.0/813.6 kB ? eta -:--:--
   ------------ --------------------------- 262.1/813.6 kB ? eta -:--:--
   ---------------------------------------- 813.6/813.6 kB 5.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import shap

In [ ]:
from timeshap.wrappers import TorchModelWrapper

class MyTorchModelWrapper(TorchModelWrapper):
    def __init__(self, model, device):
        super().__init__(model)
        self.device = device

    def predict_last_hs(self, x, y=None):
        """
        Predicts the last hidden state (or RUL value in your case) for a given input sequence.
        """
        self.model.eval()
        with torch.no_grad():
            x = torch.tensor(x, dtype=torch.float32).to(self.device)  # Convert input to tensor
            output = self.model(x).squeeze(dim=1)  # Get the RUL predictions
            return output.cpu().numpy()  # Convert predictions to NumPy


In [20]:
model_wrapped = MyTorchModelWrapper(model, device)


In [ ]:
# Convert train data into a suitable pandas DataFrame
train_data_np = np.array(train_data)  # Convert list of arrays to a single NumPy array

train_label_np = np.array([lbl.item() if hasattr(lbl, 'item') else lbl for lbl in train_label])  # Convert labels to NumPy array

# Flatten the 3D array (samples, window_size, features) into 2D format for pandas DataFrame
train_data_flat = train_data_np.reshape(-1, train_data_np.shape[2])  # Flatten along window_size

# Create a sequence ID based on how the data is structured
# Since each sequence represents one engine, generate IDs for each sequence
sequence_ids = np.repeat(np.arange(len(train_data)), train_data_np.shape[1])  # Repeat sequence IDs by window size

# Combine into a DataFrame
d_train_normalized = pd.DataFrame(
    train_data_flat,
    columns=[f"feature_{i}" for i in range(14)]  # 14 selected sensor features
)
d_train_normalized['sequence_id'] = sequence_ids  # Add inferred sequence IDs
d_train_normalized['label'] = np.repeat(train_label_np, train_data_np.shape[1])  # Repeat labels for each window

# Define model features
model_features = [f"feature_{i}" for i in range(14)]  # List of 14 sensor features

# Define sequence identifier
sequence_id_feat = "sequence_id"  # Use the generated sequence IDs


In [49]:
model_features

['feature_0',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13']

In [53]:
from timeshap.utils import calc_avg_event, calc_avg_sequence

# Calculate average event
average_event = calc_avg_event(
    d_train_normalized,
    numerical_feats=model_features,
    categorical_feats=[]
)

# Calculate average sequence
average_sequence = calc_avg_sequence(
    d_train_normalized,
    numerical_feats=model_features,
    categorical_feats=[],
    model_features=model_features,
    entity_col=sequence_id_feat
)


In [74]:
average_sequence

array([[0.39156627, 0.38064094, 0.39297772, 0.61835749, 0.25757576,
        0.16979269, 0.35119048, 0.63965885, 0.27941176, 0.20667768,
        0.39823009, 0.41666667, 0.57364341, 0.59928197],
       [0.39156627, 0.38151297, 0.39399055, 0.61674718, 0.25757576,
        0.17010679, 0.35119048, 0.63752665, 0.27941176, 0.20683249,
        0.39938438, 0.41666667, 0.57364341, 0.59831538],
       [0.39457831, 0.382385  , 0.39483457, 0.61513688, 0.27272727,
        0.17024141, 0.35119048, 0.63752665, 0.27941176, 0.2069357 ,
        0.40053867, 0.41666667, 0.57364341, 0.59707263],
       [0.39457831, 0.38325703, 0.39618501, 0.61513688, 0.27272727,
        0.17051063, 0.35714286, 0.63539446, 0.29411765, 0.20709052,
        0.4013082 , 0.41666667, 0.57364341, 0.59624413],
       [0.39457831, 0.38391105, 0.39753545, 0.61352657, 0.27272727,
        0.17069012, 0.35714286, 0.63539446, 0.29411765, 0.20734854,
        0.40246249, 0.41666667, 0.57364341, 0.59527755],
       [0.39759036, 0.38478308, 0.3

In [42]:
f_hs = lambda x, y=None: model_wrapped.predict_last_hs(x, y)

In [57]:
# Convert to NumPy and check the shape
pos_x_data = pos_x_pd[model_features].to_numpy()  # Extract features as NumPy array
print("Shape before adding batch dimension:", pos_x_data.shape)

# Add batch dimension if necessary
if pos_x_data.ndim == 2:  # If 2D, add batch dimension
    pos_x_data = np.expand_dims(pos_x_data, axis=0)  # [time_steps, num_features] -> [1, time_steps, num_features]

# Transpose to match model input shape
pos_x_data = np.transpose(pos_x_data, (0, 2, 1))  # [1, time_steps, num_features] -> [1, num_features, time_steps]

# Verify the final shape
print("Final pos_x_data shape:", pos_x_data.shape)  # Expected: [1, 14, 50]


Shape before adding batch dimension: (50, 14)
Final pos_x_data shape: (1, 14, 50)


In [58]:
# Debug input to model
print("Input to model shape:", pos_x_data.shape)

# Test model prediction
predictions = f_hs(pos_x_data)
print("Model predictions:", predictions)


Input to model shape: (1, 14, 50)
Model predictions: [112.59787]


In [59]:
pruning_dict = {'tol': 0.025}


In [60]:
event_dict = {
    'rs': 42,
    'nsamples': 32000  # Number of samples for Shapley value approximation
}


In [61]:
feature_dict = {
    'rs': 42,
    'nsamples': 32000,
    'feature_names': model_features,  # Full list of model features
    'plot_features': {feature: feature for feature in model_features[:5]}  # Top 5 features for plotting
}


In [62]:
cell_dict = {
    'rs': 42,
    'nsamples': 32000,
    'top_x_feats': 2,  # Top features to display
    'top_x_events': 2  # Top events to display
}


In [64]:
from timeshap.explainer import local_report

# Generate the local report
local_report(f_hs, pos_x_data, pruning_dict, event_dict, feature_dict, cell_dict, average_event, entity_uuid=positive_sequence_id, entity_col='all_id')



Assuming all features are model features
Provided model function fails when applied to the provided data set.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x14 and 50x50)

In [65]:
def forward(self, x):
    print(f"Input shape: {x.shape}")  # [batch_size, num_features, time_steps]

    # Temporal Convolutional Network
    xtcn = self.tcn(x)
    print(f"After TCN: {xtcn.shape}")  # Expected: [batch_size, num_channels[-1], time_steps]

    # Self-Attention
    xtcn = self.attn(xtcn)
    print(f"After Attention: {xtcn.shape}")  # Should match TCN output

    # Flatten for Fully Connected Layer
    x_flat = torch.flatten(xtcn, start_dim=1)
    print(f"After Flatten: {x_flat.shape}")  # Check if this matches fc1 input

    # Fully Connected Layers
    x_fc = self.fc1(x_flat)
    print(f"After FC1: {x_fc.shape}")

    out = self.fc2(x_fc)
    print(f"After FC2: {out.shape}")
    return out


In [43]:
f_hs

<function __main__.<lambda>(x, y=None)>

In [ ]:
from timeshap.explainer import local_report

In [25]:
# Prepare the test dataset (if not already prepared)
test_data_np = np.array(test_data)  # Convert list of arrays to a single NumPy array
test_label_np = np.array([lbl.item() if hasattr(lbl, 'item') else lbl for lbl in test_label])  # Convert labels to NumPy array
test_data_flat = test_data_np.reshape(-1, test_data_np.shape[2])  # Flatten the 3D array into 2D

# Generate sequence IDs for the test dataset
test_sequence_ids = np.repeat(np.arange(len(test_data)), test_data_np.shape[1])  # Sequence IDs
test_all_ids = [f"cycling_{i}" for i in test_sequence_ids]  # Create unique sequence identifiers

# Combine into a test DataFrame
d_test_normalized = pd.DataFrame(
    test_data_flat,
    columns=[f"feature_{i}" for i in range(14)]  # 14 selected sensor features
)
d_test_normalized['sequence_id'] = test_sequence_ids  # Add sequence IDs
d_test_normalized['all_id'] = test_all_ids  # Add unique IDs
d_test_normalized['label'] = np.repeat(test_label_np, test_data_np.shape[1])  # Add labels

# Randomly select a positive sequence ID for testing
ids_for_test = d_test_normalized['all_id'].unique()  # Get all unique sequence IDs
positive_sequence_id = f"cycling_{np.random.choice(ids_for_test)}"
pos_x_pd = d_test_normalized[d_test_normalized['all_id'] == positive_sequence_id]

# Select model features only for the chosen sequence
pos_x_data = pos_x_pd[model_features]
# Convert the selected sequence to NumPy for TimeSHAP
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)

# Define plot features (optional: select specific features to visualize)
plot_feats = model_features[:5]  # Example: select the top 5 features for plotting

# Import and prepare dictionaries for local_report
from timeshap.explainer import local_report

pruning_dict = {'tol': 0.025}  # Pruning tolerance
event_dict = {'rs': 42, 'nsamples': 32000}  # Random seed and sample count for event explanations
feature_dict = {
    'rs': 42,
    'nsamples': 32000,
    'feature_names': model_features,  # Use the list of 14 sensor features
    'plot_features': plot_feats  # Features to include in plots
}
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_feats': 2, 'top_x_events': 2}  # Top features/events to display

# Run local_report for detailed SHAP explanations
local_report(
    f_hs,  # Prediction function
    pos_x_data,  # Selected input sequence
    pruning_dict=pruning_dict,
    event_dict=event_dict,
    feature_dict=feature_dict,
    cell_dict=cell_dict,
    avg_event=average_event
)


NameError: name 'f_hs' is not defined

In [22]:
!pip uninstall shap -y



Found existing installation: shap 0.39.0
Uninstalling shap-0.39.0:
  Successfully uninstalled shap-0.39.0


In [27]:
!pip install timeshap --upgrade



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
!pip install shap==0.37.0




  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for shap: filename=shap-0.37.0-cp39-cp39-win_amd64.whl size=376154 sha256=d3919d6053969220cb0e4df47363b772fb1e328149065d883f85172555231855
  Stored in directory: c:\users\bouch\appdata\local\pip\cache\wheels\f0\55\bd\dc7c7d076ab09b5f520c07f2ed00b7890eaf71eaeccbabb37d
Successfully built shap
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.7
    Uninstalling slicer-0.0.7:
      Successfully uninstalled slicer-0.0.7



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
f_hs = lambda x, y=None: model_wrapped.predict_last_hs(x, y)


In [67]:
import numpy as np
import pandas as pd
import torch
from timeshap.wrappers import TorchModelWrapper
from timeshap.utils import calc_avg_event, calc_avg_sequence
from timeshap.explainer import local_report
from timeshap.explainer import local_pruning
from timeshap.plot import plot_temp_coalition_pruning

# =============================
# Step 1: Prepare the Test Dataset
# =============================
test_data_np = np.array(test_data)  # Convert list of arrays to a single NumPy array
test_label_np = np.array([lbl.item() if hasattr(lbl, 'item') else lbl for lbl in test_label])  # Convert labels to NumPy array
test_data_flat = test_data_np.reshape(-1, test_data_np.shape[2])  # Flatten the 3D array into 2D

# Generate sequence IDs and unique identifiers for the test dataset
test_sequence_ids = np.repeat(np.arange(len(test_data)), test_data_np.shape[1])  # Sequence IDs
test_all_ids = [f"cycling_{i}" for i in test_sequence_ids]  # Create unique sequence identifiers

# Combine into a DataFrame
d_test_normalized = pd.DataFrame(
    test_data_flat,
    columns=[f"feature_{i}" for i in range(14)]  # Replace with your sensor features
)
d_test_normalized['sequence_id'] = test_sequence_ids  # Add sequence IDs
d_test_normalized['all_id'] = test_all_ids  # Add unique IDs
d_test_normalized['label'] = np.repeat(test_label_np, test_data_np.shape[1])  # Add labels

# =============================
# Step 2: Prepare Average Event
# =============================
# Assuming `d_train_normalized`, `model_features`, and `sequence_id_feat` are already defined
average_event = calc_avg_event(
    d_train_normalized,  # Replace with your training data DataFrame
    numerical_feats=model_features,
    categorical_feats=[]
)

# =============================
# Step 3: Wrap the Model
# =============================
class TorchModelWrapper:
    def __init__(self, model):
        self.model = model

    def predict_last_hs(self, x, y=None):
        self.model.eval()
        with torch.no_grad():
            # Convert input to PyTorch tensor
            x = torch.tensor(x, dtype=torch.float32).to(device)
            # Perform forward pass
            output = self.model(x).squeeze(dim=1)
            # Convert to NumPy for TimeSHAP
            return output.cpu().numpy()

# Wrap the model
model_wrapped = TorchModelWrapper(model)

# Define `f_hs`
f_hs = lambda x, y=None: model_wrapped.predict_last_hs(x, y)

# =============================
# Step 4: Select a Positive Sequence
# =============================
# Randomly select a sequence ID for testing
ids_for_test = d_test_normalized['all_id'].unique()
positive_sequence_id = np.random.choice(ids_for_test)  # Randomly choose one sequence

# Filter the test dataset for the selected sequence
pos_x_pd = d_test_normalized[d_test_normalized['all_id'] == positive_sequence_id]

# Select model features only and convert to NumPy
pos_x_data = pos_x_pd[model_features]
pos_x_data = np.expand_dims(pos_x_data.to_numpy().copy(), axis=0)
# pos_x_data.shape => [1, 50, 14] actuellement
pos_x_data = np.transpose(pos_x_data, (0, 2, 1))
# Maintenant => [1, 14, 50]

# =============================
# Step 5: Run `local_report`
# =============================
# Define dictionaries for pruning, events, features, and cells
pruning_dict = {'tol': 0.025}  # Pruning tolerance
event_dict = {'rs': 42, 'nsamples': 32000}  # Random seed and number of samples for events
feature_dict = {
    'rs': 42,
    'nsamples': 32000,
    'feature_names': model_features,
    'plot_features': {feature: feature for feature in model_features},  # All features mapped
}
cell_dict = {'rs': 42, 'nsamples': 32000, 'top_x_feats': 2, 'top_x_events': 2}  # Top features and events to show

# Run local_report
local_report(f_hs, pos_x_data, pruning_dict, event_dict, feature_dict, cell_dict, average_event, entity_uuid=positive_sequence_id, entity_col='all_id')



Assuming all features are model features
Provided model function fails when applied to the provided data set.


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x14 and 50x50)

In [45]:
print("Input to model shape:", pos_x_data.shape)

Input to model shape: (1, 50, 14)
